In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from urllib.request import urlretrieve
import os

In [2]:
soup = BeautifulSoup(requests.get("https://download.geofabrik.de/europe/germany.html").content)

In [3]:
subregions = ["https://download.geofabrik.de/europe/" + subregion["href"] for subregion in soup.find_all("a", href = re.compile("germany/.*\.html"))]

In [10]:
lookup_table = pd.DataFrame(columns = ["subregion", "subsubregion", "path", "link"])

In [5]:
storage_directory = "/pfs/work7/workspace/scratch/tu_zxobe27-ds_project/data/OSM/raw"

In [40]:
dl_links

['berlin-latest.osm.pbf']

In [59]:
[{"subregion": re.search(r"^.*(?=/)|^.*(?=-latest)", dl_link).group(0),
                                           "subsubregion": re.search(r"(?<=/)*.*(?=-regbez)*(?=-latest)", dl_link).group(0),
                                           "link": f"https://download.geofabrik.de/europe/germany/{dl_link}"} for dl_link in dl_links]

[{'subregion': 'berlin',
  'subsubregion': 'berlin',
  'link': 'https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf'}]

In [90]:
subregions = subregions[1:]

In [91]:
os.makedirs(f"{storage_directory}/{re.search(r'^.*(?=/)|^.*(?=-latest)', dl_link).group(0)}", exist_ok = True)

In [11]:
# iterate the subregions
for subregion in subregions:
    add_subregion_path = False
    
    # get the page for the subregion
    soup = BeautifulSoup(requests.get(subregion).content)
    
    # check if there are no subsubregions
    if (len(soup.find_all("p", string = re.compile("No sub regions are defined for this region."))) > 0):
        add_subregion_path = True
        # add the entire subregion to the download query
        dl_links = [soup.find("a", href = re.compile(".*latest\.osm\.pbf$"))["href"]]
        
    else:
        # add all subsubregions to the download query
        dl_links = [subsubregion["href"] for subsubregion in soup.find_all("a", href = re.compile(".*/.*\.osm\.pbf$"))]
        
        
    # add the links to the lookup table
    lookup_table = pd.concat([lookup_table,
                            pd.DataFrame([{"subregion": re.search(r"^.*(?=/)|^.*(?=-latest)", dl_link).group(0),
                                           "subsubregion": re.search(r"(?<=/)*.*(?=-regbez)*(?=-latest)", dl_link).group(0),
                                           "path": f"{storage_directory}/{re.search(r'^.*(?=/)|^.*(?=-latest)', dl_link).group(0)}/{dl_link}",
                                           "link": f"https://download.geofabrik.de/europe/germany/{dl_link}"} for dl_link in dl_links])])
    
    # create the directory for the subsubregion
    os.makedirs(f"{storage_directory}/{re.search(r'^.*(?=/)|^.*(?=-latest)', dl_links[0]).group(0)}", exist_ok = True)
    
    # iterate over download links
    #for dl_link in dl_links:
    #    # download the file
    #    if add_subregion_path:
    #        urlretrieve(f"https://download.geofabrik.de/europe/germany/{dl_link}",
    #                        f"{storage_directory}/{re.search(r'^.*(?=/)|^.*(?=-latest)', dl_link).group(0)}/{dl_link}")
    #    else:
    #        urlretrieve(f"https://download.geofabrik.de/europe/germany/{dl_link}",
    #                        f"{storage_directory}/{dl_link}")
    
    lookup_table.to_csv(f"{storage_directory}/lookup.csv")

In [12]:
lookup_table

,subregion,subsubregion,path,link
0,baden-wuerttemberg,baden-wuerttemberg/freiburg-regbez,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
1,baden-wuerttemberg,baden-wuerttemberg/karlsruhe-regbez,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
2,baden-wuerttemberg,baden-wuerttemberg/stuttgart-regbez,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
3,baden-wuerttemberg,baden-wuerttemberg/tuebingen-regbez,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
0,bayern,bayern/mittelfranken,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
1,bayern,bayern/niederbayern,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
2,bayern,bayern/oberbayern,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
3,bayern,bayern/oberfranken,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
4,bayern,bayern/oberpfalz,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...
5,bayern,bayern/schwaben,/pfs/work7/workspace/scratch/tu_zxobe27-ds_pro...,https://download.geofabrik.de/europe/germany/b...


In [ ]:
urlretrieve(f"https://download.geofabrik.de/europe/germany/{level}.json",
                        f"{self.storage_directory}/gadm41_DEU_{level}.json")